In [ ]:
from pynq.overlays.base import BaseOverlay

# Load the base overlay
base = BaseOverlay("base.bit")

# Access LEDs directly from the base overlay
# The base overlay already has LED objects created
base.leds[0].on()   # Turn on LED 0
base.leds[1].on()   # Turn on LED 1
base.leds[2].on()   # Turn on LED 2
base.leds[3].on()   # Turn on LED 3

In [ ]:
from pynq.overlays.base import BaseOverlay
import time

# Load the base hardware design
base = BaseOverlay("base.bit")

print("Press BTN0 to start the test...")
while base.buttons[0].read() == 0:
    pass

print("WAIT FOR IT...")
# Random delay, then flash LED
time.sleep(2) 
base.leds[0].on()

start = time.time()
while base.buttons[0].read() == 0:
    pass
end = time.time()

base.leds[0].off()
print(f"Reaction time: {(end - start)*1000:.2f} ms")

In [ ]:
import time
import numpy as np
from pynq import Overlay
from pynq.lib.video import VideoMode

# --- 1. System Setup ---
base = Overlay("base.bit")
hdmi_out = base.video.hdmi_out

# Configure HDMI to 720p (24-bit color)
# If colors look garbled (rainbow strips), change '24' to '32' and update array shapes to (N, 4)
mode = VideoMode(1280, 720, 24) 
hdmi_out.configure(mode)
hdmi_out.start()

print(f"HDMI Initialized: {hdmi_out.mode}")

# --- 2. Configuration ---
WIDTH = 1280
HEIGHT = 720
NUM_PARTICLES = 2500
SPEED = 3.0
DECAY_AMOUNT = 5  # Integer fade amount (0-255)

# --- COLOR PALETTE FIX ---
# 1. Define colors in standard Hex/RGB first for sanity.
# GitHub Palette: [Darkest, Dark, Light, Neon]
HEX_COLORS = [
    [13, 17, 23],    # Background (Dark) - #0d1117
    [14, 68, 41],    # #0e4429
    [0, 109, 50],    # #006d32
    [38, 166, 65],   # #26a641
    [57, 211, 83]    # #39d353
]

# 2. Convert to NumPy array
palette_rgb = np.array(HEX_COLORS, dtype=np.uint8)

# 3. SWAP CHANNELS MANUALLY
# PYNQ raw HDMI buffers usually interpret bytes as B-G-R.
# If your screen is Blue/Red, toggle this ::-1 slice.
# Current setting: Converts RGB -> BGR
palette_final = palette_rgb[:, ::-1] 

# Separate Background color from Particle colors
BG_COLOR = palette_final[0]          # The dark background
PARTICLE_COLORS = palette_final[1:]  # The greens

# --- 3. Particle State ---
P = np.random.rand(NUM_PARTICLES, 2) * [WIDTH, HEIGHT]
# Assign random color indices (0 to 3)
color_indices = np.random.randint(0, 4, NUM_PARTICLES)
p_colors = PARTICLE_COLORS[color_indices] # Pre-lookup colors for speed

# --- 4. Loop ---
try:
    frame_out = hdmi_out.newframe()
    
    # Initialize background
    frame_out[:] = BG_COLOR
    
    print("Running... Press Ctrl+C to stop.")
    
    while True:
        # --- A. Fade Effect (Integer Math) ---
        # We want to fade towards BG_COLOR, not pure black.
        # Logic: If pixel > BG, subtract; if pixel < BG, add (simplified to just subtract for fade)
        # Using simple integer subtraction for performance on ARM
        
        # Create a view of the frame as 1D or flattened to speed up the decay math if needed,
        # but numpy vectorization on the 3D array is usually okay for 720p.
        # We use np.maximum to ensure we don't underflow below 0 (or the BG color level).
        
        # Fast Fade: Subtract DECAY_AMOUNT from all channels, clipping at 0
        frame_out[:] = np.maximum(frame_out.astype(np.int16) - DECAY_AMOUNT, 0).astype(np.uint8)
        
        # --- B. Physics (Vectorized) ---
        # 1. Flow Field Calculation
        # v = [cos(angle), sin(angle)]
        # angle = x * scale + y * scale
        scale = 0.005
        angles = np.sin(P[:, 0] * scale) + np.cos(P[:, 1] * scale)
        
        vx = np.cos(angles) * SPEED
        vy = np.sin(angles) * SPEED
        
        # 2. Update Positions
        P[:, 0] += vx
        P[:, 1] += vy
        
        # 3. Boundary Wrapping (Pac-man style is faster than respawning for flow fields)
        P[:, 0] = np.mod(P[:, 0], WIDTH)
        P[:, 1] = np.mod(P[:, 1], HEIGHT)
        
        # --- C. Rendering ---
        # 1. Cast positions to integers
        ix = P[:, 0].astype(np.int32)
        iy = P[:, 1].astype(np.int32)
        
        # 2. Filter valid indices (just in case mod failed or edges)
        # Note: PYNQ buffer crashes HARD if you write out of bounds.
        mask = (ix >= 0) & (ix < WIDTH) & (iy >= 0) & (iy < HEIGHT)
        
        valid_x = ix[mask]
        valid_y = iy[mask]
        valid_c = p_colors[mask]
        
        # 3. Direct Buffer Write
        # This writes BGR (or RGB) values directly into the memory mapped frame
        frame_out[valid_y, valid_x] = valid_c
        
        # --- D. Output ---
        hdmi_out.writeframe(frame_out)

except KeyboardInterrupt:
    print("Stopping...")

finally:
    hdmi_out.stop()
    del hdmi_out
    del base